## Import Libraries and load data

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor, plot_importance
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Paths
# df = pd.read_csv("Proprocessed_Dataset.csv")  # update this to your notebook's directory if needed
df = pd.read_csv("C:/Users/ianta/Desktop/Ian/University (NUS)/Y4S1/BT4103 Business Analytics Capstone Project/Github Repo/Proprocessed_Dataset_2.csv")
df.head()

In [ ]:
# Drop index column, if present
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0", "OPERATION_ID"])

df.head()

In [33]:
MODEL_DIR = Path("xgb_models")
MODEL_DIR.mkdir(exist_ok=True)

# Random seed
RANDOM_STATE = 42

In [34]:
import sys, xgboost
print(sys.executable)
print(xgboost.__version__)

c:\Users\ianta\anaconda3\python.exe
3.0.0


In [35]:
import optuna

def frequency_encode(df, col):
    freq = df[col].value_counts() / len(df)
    df[col] = df[col].map(freq)
    return df

def target_encode(train, test, col, target):
    means = train.groupby(col)[target].mean()
    train[col] = train[col].map(means)
    test[col] = test[col].map(means)
    test[col] = test[col].fillna(train[target].mean())
    return train, test

# === 3. Define utility function to train and evaluate XGBoost model ===
def train_xgboost_for_target(df, target_col):
    print(f"\n=== Predicting {target_col} ===")

    # ---------------------------------------------------------------------
    # STEP 0: Handle per-SURGICAL_CODE grouping logic
    # ---------------------------------------------------------------------
    if 'SURGICAL_CODE' in df.columns:
        print("\n=== Grouped Training by SURGICAL_CODE ===")

        # Count frequency
        code_counts = df['SURGICAL_CODE'].value_counts()
        df['SURGICAL_CODE_CONFIDENCE'] = df['SURGICAL_CODE'].map(code_counts)

        # Split common vs rare
        common_codes = code_counts[code_counts >= 100].index
        # common_codes = []
        rare_codes = code_counts[code_counts < 100].index

        print(f"✓ Common codes (≥100 cases): {len(common_codes)}")     
        print(f"✓ Rare codes (<100 cases): {len(rare_codes)}")

        # Train one model per common code
        group_models = {}
        codes_to_move = []

        for code in common_codes:
            subset = df[df['SURGICAL_CODE'] == code].copy()
            drop_cols = [
                'SURGICAL_CODE', 
                'SURGICAL_CODE_MEDIAN_SURGERY', 
                'SURGICAL_CODE_MEDIAN_USAGE', 
                'SURGICAL_CODE_MEDIAN_CONFIDENCE'
            ]
            subset = subset.drop(columns=[c for c in drop_cols if c in subset.columns])

            print(f"\n--- Training model for SURGICAL_CODE={code} ({len(subset)} rows) ---")
            model, imp, Xtr, Xte, ytr, yte, metrics = _train_xgb_core(
                subset, target_col, suffix=f"_surgicalcode_{code}", n_trials=15, tune=False
            )

            if metrics["overfit_flag"]:
                codes_to_move.append(code)
            else:
                group_models[code] = model

        # Move overfitted codes to rare_codes (for retraining in OTHERS)
        if codes_to_move:
            print(f"\n⚠️ Moving {len(codes_to_move)} overfitted codes to 'OTHERS': {codes_to_move}")
            rare_codes = rare_codes.union(codes_to_move)

        # Train the “OTHERS” model (rare + overfitted)
        subset_others = df[df['SURGICAL_CODE'].isin(rare_codes)].copy()
        print(f"\n--- Training 'OTHERS' model ({len(subset_others)} rows) ---")
        model_others, imp_others, Xtr_o, Xte_o, ytr_o, yte_o, metrics_others = _train_xgb_core(
            subset_others, target_col, suffix="_surgicalcode_OTHERS", n_trials=20, tune=False
        )
        group_models['OTHERS'] = model_others

        print("\n✓ Completed training for all SURGICAL_CODE groups (including OTHERS)")
        return group_models  # return dictionary of models

    # ---------------------------------------------------------------------
    # STEP 1: If no SURGICAL_CODE column, do normal training
    # ---------------------------------------------------------------------
    else:
        print("No SURGICAL_CODE column found — running standard training.")
        model, imp, Xtr, Xte, ytr, yte = _train_xgb_core(df, target_col)
        return model, imp, Xtr, Xte, ytr, yte

def _train_xgb_core(df, target_col, suffix="", n_trials=20, overfit_threshold=0.3, tune=True):
    """
    Core XGBoost training routine.
    If tune=True → uses Optuna hyperparameter tuning
    If tune=False → uses default XGBoost parameters
    """

    df_model = df.dropna(subset=[target_col]).copy()
    X = df_model.drop(columns=[target_col])
    y = df_model[target_col]

    other_target = (
        "ACTUAL_USAGE_DURATION" if target_col == "ACTUAL_SURGERY_DURATION"
        else "ACTUAL_SURGERY_DURATION"
    )
    if other_target in X.columns:
        X = X.drop(columns=[other_target])

    # Handle categoricals
    cat_cols = X.select_dtypes(include=["object"]).columns
    high_card_cols = [c for c in cat_cols if X[c].nunique() > 50]
    low_card_cols  = [c for c in cat_cols if X[c].nunique() <= 50]

    for c in high_card_cols:
        X = frequency_encode(X, c)
    X = pd.get_dummies(X, columns=low_card_cols, drop_first=True)

    X.columns = (
        X.columns.astype(str)
        .str.replace('[', '(', regex=False)
        .str.replace(']', ')', regex=False)
        .str.replace('<', '_lt_', regex=False)
        .str.replace('>', '_gt_', regex=False)
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE
    )

    # === Use Optuna or Default XGB ===
    if tune:
        print("Tuning hyperparameters with Optuna...")
        def objective(trial):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'n_estimators': 2000,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.5, 5.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
                'random_state': RANDOM_STATE,
                'n_jobs': -1,
            }
            model = XGBRegressor(**params)
            rmse = -cross_val_score(
                model, X_train, y_train,
                scoring='neg_root_mean_squared_error',
                cv=3, n_jobs=-1
            ).mean()
            return rmse

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        best_params = study.best_params
        print("Best parameters:", best_params)
    else:
        print("Skipping Optuna tuning — using default XGBoost parameters.")
        best_params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }

    # === Train final model ===
    model = XGBRegressor(
        **best_params,
        n_estimators=3000,
        early_stopping_rounds=200,
    )
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    # === Evaluate ===
    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)

    rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
    mae_train  = mean_absolute_error(y_train, y_pred_train)
    r2_train   = r2_score(y_train, y_pred_train)
    rmse_test  = mean_squared_error(y_test, y_pred_test, squared=False)
    mae_test   = mean_absolute_error(y_test, y_pred_test)
    r2_test    = r2_score(y_test, y_pred_test)

    # --- MAPE metrics ---
    def safe_mape(actual, fitted):
        mask = (actual != 0)
        return np.mean(np.abs((actual[mask] - fitted[mask]) / actual[mask]))
    def safe_mape_fitted(actual, fitted):
        mask = (fitted != 0)
        return np.mean(np.abs((actual[mask] - fitted[mask]) / fitted[mask]))

    mape_train_actual = safe_mape(y_train.values, y_pred_train) * 100
    mape_train_fitted = safe_mape_fitted(y_train.values, y_pred_train) * 100
    mape_test_actual  = safe_mape(y_test.values, y_pred_test) * 100
    mape_test_fitted  = safe_mape_fitted(y_test.values, y_pred_test) * 100

    rel_gap = (mape_test_actual - mape_train_actual) / max(mape_train_actual, 1e-6)

    # --- Overfitting detection ---
    overfit_flag = (
        (rel_gap > overfit_threshold) or 
        (mape_test_fitted > 35) or 
        (r2_test < 0.1)
    )

    if overfit_flag:
        print(f"⚠️ Overfitting detected — conditions triggered:")
        if rel_gap > overfit_threshold:
            print(f"   • Test MAPE is {rel_gap*100:.1f}% higher than Train (>{overfit_threshold*100:.0f}%)")
        if mape_test_fitted > 35:
            print(f"   • MAPE (Fitted) exceeds 35% → {mape_test_fitted:.1f}%")
        if r2_test < 0.1:
            print(f"   • R² below acceptable threshold (0.1) → {r2_test:.3f}")
        print("→ Reassigning this SURGICAL_CODE to 'OTHERS' model.")

    print(f"Train: RMSE={rmse_train:.2f}, MAE={mae_train:.2f}, "
          f"MAPE (Actual): {mape_train_actual:.1f}%, MAPE (Fitted): {mape_train_fitted:.1f}%, R²={r2_train:.3f}")
    print(f" Test: RMSE={rmse_test:.2f}, MAE={mae_test:.2f}, "
          f"MAPE (Actual): {mape_test_actual:.1f}%, MAPE (Fitted): {mape_test_fitted:.1f}%, R²={r2_test:.3f}")

    # === Feature importance ===
    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    importance.to_csv(MODEL_DIR / f"feature_importance_{target_col}{suffix}.csv", index=False)

    model.save_model(MODEL_DIR / f"model_{target_col}{suffix}.json")

    return model, importance, X_train, X_test, y_train, y_test, {
        "MAPE_train_actual": mape_train_actual,
        "MAPE_test_actual": mape_test_actual,
        "rel_gap": rel_gap,
        "overfit_flag": overfit_flag
    }

In [36]:
# Remove columns that are relating to USAGE, when training for SURGERY
usage_columns = [col for col in df.columns if 'USAGE' in col]
df_surgery = df.drop(columns=usage_columns)

model_surgery, imp_surgery, X_train_surgery, X_test_surgery, y_train_surgery, y_test_surgery = train_xgboost_for_target(df_surgery, "ACTUAL_SURGERY_DURATION")


=== Predicting ACTUAL_SURGERY_DURATION ===

=== Grouped Training by SURGICAL_CODE ===
✓ Common codes (≥100 cases): 338
✓ Rare codes (<100 cases): 2787

--- Training model for SURGICAL_CODE=SF702C (32108 rows) ---
Skipping Optuna tuning — using default XGBoost parameters.
Train: RMSE=15.29, MAE=11.08, MAPE (Actual): 38.4%, MAPE (Fitted): 33.0%, R²=0.368
 Test: RMSE=15.73, MAE=11.55, MAPE (Actual): 41.1%, MAPE (Fitted): 34.6%, R²=0.266

--- Training model for SURGICAL_CODE=SF701I (19716 rows) ---
Skipping Optuna tuning — using default XGBoost parameters.
⚠️ Overfitting detected — conditions triggered:
   • MAPE (Fitted) exceeds 35% → 38.6%
→ Reassigning this SURGICAL_CODE to 'OTHERS' model.
Train: RMSE=7.67, MAE=4.94, MAPE (Actual): 42.9%, MAPE (Fitted): 36.9%, R²=0.304
 Test: RMSE=9.29, MAE=5.29, MAPE (Actual): 44.9%, MAPE (Fitted): 38.6%, R²=0.184

--- Training model for SURGICAL_CODE=SP834U (6339 rows) ---
Skipping Optuna tuning — using default XGBoost parameters.
Train: RMSE=11.30, 

ValueError: too many values to unpack (expected 6)

In [ ]:
plt.figure(figsize=(9, 10))
plot_importance(model_surgery, importance_type="gain", max_num_features=20) 
plt.title("Top 20 Feature Importances (Gain) (SURGERY)")
plt.tight_layout()
plt.show()

In [ ]:
# === 3. Compute predictions and residuals ===
y_pred_train_surgery = model_surgery.predict(X_train_surgery)
y_pred_test_surgery  = model_surgery.predict(X_test_surgery)

resid_train_surgery = y_train_surgery - y_pred_train_surgery
resid_test_surgery  = y_test_surgery - y_pred_test_surgery

# === 4. Plot residuals vs fitted for train ===
plt.figure(figsize=(7, 5))
sns.scatterplot(x=y_pred_train_surgery, y=resid_train_surgery, alpha=0.6)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Fitted values (Train)")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Train)")
plt.tight_layout()
plt.show()

# === 5. Plot residuals vs fitted for test ===
plt.figure(figsize=(7, 5))
sns.scatterplot(x=y_pred_test_surgery, y=resid_test_surgery, alpha=0.6)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Fitted values (Test)")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Test)")
plt.tight_layout()
plt.show()

In [ ]:
# === Run for ACTUAL_USAGE_DURATION ===

# print(df.columns)

# Remove columns that are relating to SURGERY, when training for USAGE
surgery_columns = [col for col in df.columns if 'SURGERY' in col]
df_usage = df.drop(columns=surgery_columns)

# print(df_usage.columns)

model_usage, imp_usage, X_train_usage, X_test_usage, y_train_usage, y_test_usage = train_xgboost_for_target(df_usage, "ACTUAL_USAGE_DURATION")
model_usage.save_model("xgb_models/model_usage.json")

In [ ]:
plt.figure(figsize=(9, 10))
plot_importance(model_usage, importance_type="gain", max_num_features=20)
plt.title("Top 20 Feature Importances (Gain) (USAGE)")
plt.tight_layout()
plt.show()

In [ ]:
# === 3. Compute predictions and residuals ===
y_pred_train_usage = model_usage.predict(X_train_usage)
y_pred_test_usage  = model_usage.predict(X_test_usage)

resid_train_usage = y_train_usage - y_pred_train_usage
resid_test_usage  = y_test_usage - y_pred_test_usage

# === 4. Plot residuals vs fitted for train ===
plt.figure(figsize=(7, 5))
sns.scatterplot(x=y_pred_train_usage, y=resid_train_usage, alpha=0.6)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Fitted values (Train)")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Train)")
plt.tight_layout()
plt.show()

# === 5. Plot residuals vs fitted for test ===
plt.figure(figsize=(7, 5))
sns.scatterplot(x=y_pred_test_usage, y=resid_test_usage, alpha=0.6)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Fitted values (Test)")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Test)")
plt.tight_layout()
plt.show()